In [1]:
import networkx as nx
import urllib.request
import json
import pandas as pd
import re
import pickle
import numpy as np

Now that we cleaned up the data we can build the network.

In [ ]:
data = np.load()
author_posts = dict()
for post in data:
    # Create a dictionary of authors and theirflair, who they answered and their comment
    author_name = re.findall(r'"out_going":"(.*?)"',post)
    author_flair = re.findall(r'"author_flair_text":"(.*?)"',post)
    answered = re.findall(r'"in_going":"(.*?)"',post)
    comment = re.findall(r'"comment":"(.*?)"',post)

    # Add author to dictionary or add attributes
    if author_name not in author_posts:
        info_so_far = author_posts[author_name]
        author_posts[author_name] = [info_so_far[0],info_so_far[1].append(answered),info_so_far[2]+comment]
    else:
        author_posts[author_name] = [author_flair,[answered],comment]

# Create a directed graph
DirectedGraph = nx.DiGraph()
for author_name in author_posts:
    # Add nodes to the graph
    DirectedGraph.add_node(author_name,flair=author_posts[author_name][0],comments=author_posts[author_name][2])

    # Add edges
    for edge_to in author_posts[author_name][1]:
        DirectedGraph.add_edge(author_name,edge_to)

# Save authors as a pickle file
pickle.dump(DirectedGraph, open('Directed2022.pkl','wb'))